# Cluster Analysis

## Imports, Functions, & Settings

In [44]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from glob import glob
from zipfile import ZipFile
import shutil

## Data Loading

In [54]:
df_dir = "../data/dataframes/"

In [46]:
# Extract the BOLD signal data
with ZipFile(df_dir + "BOLD.zip") as zfile:
    zfile.extractall(df_dir)
BOLD_csv_l = glob(df_dir + "BOLD/" + "*.csv", recursive=True)
BOLD_df_l = []
for csv in BOLD_csv_l:
    BOLD_df_l.append(pd.read_csv(csv))
shutil.rmtree(df_dir + "BOLD")

In [61]:
# Load fALFF, CV, & subject names
fALFF_df = pd.read_csv(df_dir + "fALFF.csv", index_col=0)
cv_df = pd.read_csv(df_dir + "cv_df.csv", index_col=0)
participant_df = pd.read_csv(df_dir + "participants.tsv", sep="\t")

In [62]:
participant_df.head()

,participant_id,age,sex,handedness,years_of_education
0,sub-0202,76.9260273972603,1,3 Right,16
1,sub-0203,73.3257654016019,1,2 Left,12
2,sub-0204,55.0602739726027,1,3 Right,14
3,sub-0205,77.43231529306091,1,3 Right,16
4,sub-0206,72.0191780821918,1,3 Right,12


In [48]:
fALFF_df.head()

,Frontal Pole,Insular Cortex,Superior Frontal Gyrus,Middle Frontal Gyrus,"Inferior Frontal Gyrus, pars triangularis","Inferior Frontal Gyrus, pars opercularis",Precentral Gyrus,Temporal Pole,"Superior Temporal Gyrus, anterior division","Superior Temporal Gyrus, posterior division",...,Right Cerebral White Matter,Right Cerebral Cortex,Right Lateral Ventricle,Right Thalamus,Right Caudate,Right Putamen,Right Pallidum,Right Hippocampus,Right Amygdala,Right Accumbens
Subject,,,,,,,,,,,,,,,,,,,,,
sub-0270,0.011413,0.010983,0.010253,0.010471,0.011631,0.010933,0.011267,0.008058,0.008130,0.010452,...,0.012502,0.012156,0.009588,0.011064,0.009658,0.010349,0.009297,0.008296,0.008801,0.010626
sub-0205,0.008063,0.008100,0.008432,0.008537,0.008519,0.008345,0.008678,0.007926,0.008483,0.008496,...,0.007980,0.008252,0.007906,0.007987,0.007816,0.008021,0.008095,0.008172,0.009426,0.009352
sub-0231,0.010883,0.010724,0.013460,0.011124,0.011169,0.011045,0.013046,0.007863,0.010838,0.008925,...,0.012944,0.012541,0.007646,0.009958,0.009135,0.009343,0.009173,0.008074,0.007594,0.006212
sub-0240,0.012848,0.011475,0.012581,0.012059,0.011521,0.010469,0.012200,0.009404,0.009567,0.009665,...,0.012735,0.012402,0.007286,0.006726,0.008681,0.008327,0.008582,0.008873,0.006382,0.008044
sub-0238,0.010721,0.008784,0.009308,0.009108,0.010489,0.010703,0.008949,0.007983,0.009652,0.009742,...,0.012790,0.010455,0.008811,0.008961,0.006450,0.008073,0.008078,0.009224,0.008114,0.007508


In [49]:
cv_df.head()

,Frontal Pole,Insular Cortex,Superior Frontal Gyrus,Middle Frontal Gyrus,"Inferior Frontal Gyrus, pars triangularis","Inferior Frontal Gyrus, pars opercularis",Precentral Gyrus,Temporal Pole,"Superior Temporal Gyrus, anterior division","Superior Temporal Gyrus, posterior division",...,Right Cerebral White Matter,Right Cerebral Cortex,Right Lateral Ventricle,Right Thalamus,Right Caudate,Right Putamen,Right Pallidum,Right Hippocampus,Right Amygdala,Right Accumbens
Subject,,,,,,,,,,,,,,,,,,,,,
sub-0270,0.007361,0.004123,0.003377,0.003526,0.004109,0.004509,0.003287,0.006441,0.004820,0.004280,...,0.003499,0.003508,0.003367,0.005808,0.012133,0.007270,0.016889,0.013879,0.026816,0.071313
sub-0205,0.015797,0.018319,0.022378,0.019799,0.019887,0.016706,0.019045,0.018231,0.020285,0.016501,...,0.023049,0.019843,0.049101,0.054028,0.060103,0.018755,0.014706,0.025465,0.028863,0.070590
sub-0231,0.004760,0.004682,0.006065,0.004987,0.006065,0.006556,0.006051,0.003977,0.005636,0.005841,...,0.004963,0.005630,0.005002,0.006167,0.011677,0.002962,0.006408,0.005817,0.019217,0.017145
sub-0240,0.006936,0.007587,0.006876,0.006616,0.007496,0.006326,0.005463,0.010617,0.007100,0.008672,...,0.005357,0.006636,0.006510,0.006285,0.007420,0.004251,0.005474,0.006373,0.011001,0.018198
sub-0238,0.004008,0.003126,0.003351,0.003580,0.004475,0.003954,0.003441,0.007030,0.003373,0.003251,...,0.003166,0.004256,0.003354,0.004243,0.005806,0.002414,0.004007,0.004253,0.009129,0.006102


In [50]:
subject_name_df.head()

,Name
0,sub-0270
1,sub-0205
2,sub-0231
3,sub-0240
4,sub-0238


In [51]:
len(BOLD_df_l)

58

In [52]:
BOLD_df_l[0].head()

,Frontal Pole,Insular Cortex,Superior Frontal Gyrus,Middle Frontal Gyrus,"Inferior Frontal Gyrus, pars triangularis","Inferior Frontal Gyrus, pars opercularis",Precentral Gyrus,Temporal Pole,"Superior Temporal Gyrus, anterior division","Superior Temporal Gyrus, posterior division",...,Right Cerebral White Matter,Right Cerebral Cortex,Right Lateral Ventricle,Right Thalamus,Right Caudate,Right Putamen,Right Pallidum,Right Hippocampus,Right Amygdala,Right Accumbens
0,4628.117925,4746.306075,5486.574830,5088.290530,5440.212500,5218.093333,5103.954315,4216.573034,5430.396552,4903.854015,...,5189.195873,5145.633034,6815.800000,6358.303279,6373.383333,3400.520548,4156.740741,4109.000000,2906.482759,4936.8
1,3691.445165,3884.212617,4181.600340,4227.914928,4235.154167,4154.226667,4046.993655,2968.114607,4064.750000,3779.284672,...,3879.044317,3951.924459,3698.295238,3583.942623,3464.183333,2918.534247,3169.666667,3081.596491,2218.310345,2736.6
2,3582.954599,3785.137850,3983.423469,4103.036918,4065.241667,4003.928889,3865.049492,2813.229213,3906.525862,3650.985401,...,3764.681326,3818.632858,3529.304762,3418.942623,3232.066667,2887.863014,3104.518519,3045.175439,2178.413793,2692.2
3,3552.377358,3758.502336,3926.698980,4064.926164,4011.850000,3955.662222,3814.277919,2770.784270,3832.732759,3619.474453,...,3720.356225,3773.511182,3429.457143,3319.024590,3131.483333,2887.246575,3096.555556,3013.701754,2182.068966,2618.2
4,3543.304245,3742.570093,3906.391156,4054.321027,4004.233333,3950.964444,3794.552030,2718.312360,3815.922414,3593.729927,...,3705.027402,3758.676880,3396.152381,3271.606557,3106.500000,2876.890411,3059.259259,2969.701754,2097.551724,2591.6


## Cluster Analysis